In [ ]:
%pip install pandas numpy matplotlib tensorflow keras

In [ ]:
%pip install xlrd

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import ticker as ticker

In [ ]:
train_data = pd.read_excel(r"./data/Data_Train.xlsx", sheet_name="Sheet1")

In [ ]:
train_data = train_data.dropna(how='any',axis=0)
train_data = train_data.to_numpy()

In [ ]:
# assume `Price` in Lakh
data_params = ['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price', 'Price']

In [ ]:
def one_hot_encode(data:np.ndarray, n:int, categories:list = None) -> np.ndarray:
    """
    Convert the given input into one-hot-encoding.
    Use the given categories, if exist then convert the data,
    else convert indices to one-hot
    """

    try:
        assert categories is None
        data = data.astype(int)
    except (ValueError, AssertionError):
        data = np.vectorize(categories.index)(data)
    finally:
        targets = np.array(data).reshape(-1)
    
    return np.eye(n)[targets][0]

In [ ]:
one_encode_params = ["Name", "Location", "Year", "Fuel_Type", "Transmission", "Owner_Type"]

def encode_params(data, params):
    ix = map(data_params.index, params) 
    encoded_params = {}

    for i in ix:
        m = data_params[i]
        encoded_params[m] = [np.unique(data[:,i].astype(str), axis=0).tolist(), None]
        encoded_params[m][1] = list(map(lambda x: one_hot_encode(x, len(encoded_params[m][0]), encoded_params[m][0]), data[:,i].astype(str)))
    
    return encoded_params

In [ ]:
categories = encode_params(train_data, one_encode_params)

In [ ]:
categories['Year']